In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import math

In [25]:
# Task Mode
'''
1. sequence reversal: 'seq_rev'
2. basic arithmetic: 'basic_arith'
3. copying task: 'copy'
4. sorting numbers: 'sort'
5. character-level text generation: 'text_gen'
'''
task_mode = 'seq_rev'

In [26]:
# dataset preparation for character-level text generation
with open('data/alice_1.txt', 'r', encoding='utf-8') as file:
    text = file.read()

chars = list(text)
char_counts = Counter(chars)

vocab = list(char_counts.keys())
vocab_size = len(vocab)
char_to_int = {char: i for i, char in enumerate(vocab)}
int_to_char = {i: char for char, i in char_to_int.items()}

SEQUENCE_LENGTH = 64
samples = [chars[i:i+SEQUENCE_LENGTH+1] for i in range(len(chars)-SEQUENCE_LENGTH)]

class TextDataset(Dataset):
    def __init__(self, samples, char_to_int):
        self.samples = samples
        self.char_to_int = char_to_int

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_seq = torch.LongTensor([self.char_to_int[word] for word in sample[:-1]])
        target_seq = torch.LongTensor([self.char_to_int[word] for word in sample[1:]])
        return input_seq, target_seq

In [27]:
# Dataset Preparation for Sorting Task
class SortingDataset(Dataset):
    def __init__(self, seq_len=6, num_samples=10000, num_range=10):
        self.seq_len = seq_len
        self.num_samples = num_samples
        self.num_range = num_range
        self.data = self.generate_data()

    def generate_data(self):
        data = []
        for _ in range(self.num_samples):
            seq = torch.randint(1, self.num_range, (self.seq_len,))
            sorted_seq = torch.sort(seq)[0]  # Sorted sequence as target
            data.append((seq, sorted_seq))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [28]:
# Dataset Preparation for Sequence Reversal Task
class SequenceReversalDataset(Dataset):
    def __init__(self, seq_len=6, num_samples=10000, num_range=10):
        self.seq_len = seq_len
        self.num_samples = num_samples
        self.num_range = num_range
        self.data = [torch.randint(1, num_range, (seq_len,)) for _ in range(num_samples)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq = self.data[idx]
        target_seq = torch.flip(input_seq, dims=[0])  # Reversed sequence as target
        return input_seq, target_seq

In [29]:
# Dataset Preparation for Copying Task
class CopyingTaskDataset(Dataset):
    def __init__(self, seq_len=6, num_samples=10000, num_range=10):
        self.seq_len = seq_len
        self.num_samples = num_samples
        self.num_range = num_range
        self.data = [torch.randint(1, num_range, (seq_len,)) for _ in range(num_samples)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq = self.data[idx]
        target_seq = input_seq.clone()  # Create a target sequence identical to the input
        return input_seq, target_seq

In [30]:
# Digits 0-9 and '+' as 10
digit = {str(i): i for i in range(10)}
digit.update({"+": 10})  # Adding operator

class BasicArithmeticDataset(Dataset):
    def __init__(self, seq_len=6, num_samples=10, num_range=10):
        self.seq_len = seq_len
        self.num_samples = num_samples
        self.num_range = num_range
        self.data = []
        self.targets = []
        
        for _ in range(num_samples):
            # Generate two random numbers and an operator
            first_number = torch.randint(1, num_range, (self.seq_len // 2,))
            second_number = torch.randint(1, num_range, (self.seq_len // 2,))
            operation = "+"  # Can be extended for other operations
            
            # Convert the operator to its corresponding integer (10)
            operation_int = digit[operation]

            # Combine into a single sequence (input)
            input_sequence = torch.cat((first_number, torch.tensor([operation_int]), second_number))
            self.data.append(input_sequence)

            # Perform the arithmetic operation and store the result (target)
            result = self.perform_operation(first_number, second_number, operation)
            self.targets.append(result)

    def perform_operation(self, first_number, second_number, operation):
        if operation == "+":
            # Convert the tensors to integers for addition
            num1 = int("".join(map(str, first_number.tolist())))
            num2 = int("".join(map(str, second_number.tolist())))
            result = num1 + num2
        
        # Convert result to a tensor of digits
        # result_tensor = torch.tensor([int(d) for d in str(result)])
        result_tensor = torch.tensor([int(d) for d in str(result)], dtype=torch.long)

        # Prepend 0 if the result has only 3 digits
        if result_tensor.size(0) < 4:  # Adjusted to check if < 4 digits
            result_tensor = torch.cat((torch.tensor([0]), result_tensor))
        
        return result_tensor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_seq = self.data[idx]
        target_seq = self.targets[idx]
        return input_seq, target_seq

# Test the BasicArithmeticDataset
dataset = BasicArithmeticDataset(seq_len=6, num_samples=10, num_range=10)

# Print out the samples and their corresponding targets
for i in range(len(dataset)):
    input_seq, target_seq = dataset[i]
    print(f"Input Sequence: {input_seq.tolist()} | Target Sequence: {target_seq.tolist()}")


Input Sequence: [7, 9, 9, 10, 7, 5, 6] | Target Sequence: [1, 5, 5, 5]
Input Sequence: [4, 5, 2, 10, 7, 5, 7] | Target Sequence: [1, 2, 0, 9]
Input Sequence: [9, 4, 3, 10, 6, 6, 6] | Target Sequence: [1, 6, 0, 9]
Input Sequence: [8, 6, 9, 10, 1, 4, 5] | Target Sequence: [1, 0, 1, 4]
Input Sequence: [9, 8, 9, 10, 8, 8, 9] | Target Sequence: [1, 8, 7, 8]
Input Sequence: [2, 8, 7, 10, 9, 5, 7] | Target Sequence: [1, 2, 4, 4]
Input Sequence: [7, 4, 2, 10, 5, 4, 6] | Target Sequence: [1, 2, 8, 8]
Input Sequence: [3, 2, 6, 10, 3, 2, 9] | Target Sequence: [0, 6, 5, 5]
Input Sequence: [9, 3, 3, 10, 6, 5, 9] | Target Sequence: [1, 5, 9, 2]
Input Sequence: [5, 5, 2, 10, 3, 9, 3] | Target Sequence: [0, 9, 4, 5]


In [31]:
# Create datasets based on task mode
if task_mode == 'text_gen':
    dataset = TextDataset(samples, char_to_int)
    output_size = vocab_size  # For text generation, output is vocab size

seq_len = 6
num_samples = 10000
num_range = 10

if task_mode == 'basic_arith':
    seq_len = 7

if task_mode == 'sort':
    dataset = SortingDataset(seq_len=seq_len, num_samples=num_samples, num_range=num_range)
    output_size = 1  # For sorting, we output a single number per element in sequence
if task_mode == 'seq_rev':
    dataset = SequenceReversalDataset(seq_len=seq_len, num_samples=num_samples, num_range=num_range)
    output_size = 1
if task_mode == 'copy':
    dataset = CopyingTaskDataset(seq_len=seq_len, num_samples=num_samples, num_range=num_range)
    output_size = 1
if task_mode == 'basic_arith':
    dataset = BasicArithmeticDataset(seq_len=seq_len, num_samples=num_samples, num_range=num_range)
    output_size = 1

In [32]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Split the indices for training and testing
def split_dataset(dataset, test_size=0.2):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_indices, test_indices = train_test_split(indices, test_size=test_size, random_state=42)
    
    train_dataset = Subset(dataset, train_indices)
    test_dataset = Subset(dataset, test_indices)
    
    return train_dataset, test_dataset

train_dataset, test_dataset = split_dataset(dataset, test_size=0.2)

In [33]:
# Create data loaders for both train and test sets
BATCH_SIZE = 32
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [34]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [35]:
# Positional Encoding Class
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model, dropout_rate=0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout_rate)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [36]:
# Transformer Model
class Transformer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads, seq_len, output_size, dropout_rate):
        super(Transformer, self).__init__()
        if task_mode == 'text_gen':
            self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim, dropout_rate=dropout_rate)
        if task_mode == 'sort' or task_mode == 'seq_rev' or task_mode == 'copy' or task_mode == 'basic_arith':
            self.pos_encoder = PositionalEncoding(max_len=seq_len, d_model=embed_dim, dropout_rate=dropout_rate)
        
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            batch_first=True,
            dropout=dropout_rate
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        if task_mode == 'text_gen':
            self.linear = nn.Linear(embed_dim, output_size)  # Adjust output size dynamically
        
        if task_mode == 'sort' or task_mode == 'seq_rev' or task_mode == 'copy' or task_mode == 'basic_arith':
            self.linear = nn.Linear(embed_dim, vocab_size)
        
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        emb = self.emb(x)
        input_mask = generate_square_subsequent_mask(x.size(1)).to(x.device)
        x = self.pos_encoder(emb)

        if task_mode == 'text_gen':
            x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        
        if task_mode == 'sort' or task_mode == 'seq_rev' or task_mode == 'copy' or task_mode == 'basic_arith':
            x = self.decoder(x, memory=x, tgt_mask=input_mask)

        x = self.dropout(x)
        out = self.linear(x)
        return out


In [37]:
# Model and Training Setup

if task_mode == 'seq_rev':
    vocab_size = num_range

if task_mode == 'copy':
    output_size = num_range

if task_mode == 'basic_arith':
    vocab_size = num_range + 2

model = Transformer(
    vocab_size=vocab_size, 
    embed_dim=100, 
    num_layers=2, 
    num_heads=2, 
    seq_len=seq_len,
    output_size=output_size,
    dropout_rate=0.2
    )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [38]:
# Training Loop
def train(model, epochs, dataloader, criterion, task_mode):
    model.train()
    for epoch in range(1, epochs + 1):
        running_loss = 0
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            
            if task_mode == 'text_gen':
                target_seq = target_seq.contiguous().view(-1)
                outputs = outputs.view(-1, vocab_size)
                loss = criterion(outputs, target_seq)
            
            if task_mode == 'sort' or task_mode == 'seq_rev':
                target_seq = target_seq.unsqueeze(1)  # Add a sequence dimension for targets
                target_seq = target_seq.view(-1)  # Flatten target for loss calculation
                outputs = outputs.view(-1, vocab_size)  # Flatten output for loss calculation
                loss = criterion(outputs, target_seq)

            if task_mode == 'copy':
                target_seq = target_seq.view(-1)
                outputs = outputs.view(-1, vocab_size)
                loss = criterion(outputs, target_seq)
            
            if task_mode == 'basic_arith':
                outputs = outputs[:, :target_seq.size(1), :]
                outputs = outputs.reshape(-1, vocab_size)  # Flatten outputs to (batch_size * seq_len, vocab_size)
                target_seq = target_seq.reshape(-1)  # Flatten target sequence
                loss = criterion(outputs, target_seq)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
        
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch}/{epochs} loss: {epoch_loss:.4f}")

# Start training
epochs = 10
train(model, epochs, train_dataloader, criterion, task_mode)

Epoch 1/10 loss: 1.1965
Epoch 2/10 loss: 0.7492
Epoch 3/10 loss: 0.6375
Epoch 4/10 loss: 0.5239
Epoch 5/10 loss: 0.4379
Epoch 6/10 loss: 0.3807
Epoch 7/10 loss: 0.3339
Epoch 8/10 loss: 0.2975
Epoch 9/10 loss: 0.2772
Epoch 10/10 loss: 0.2482


In [39]:
# Inference: For Text Generation Task
def return_int_vector(text):
    chars = list(text)
    input_seq = torch.LongTensor([char_to_int[char] for char in chars[-SEQUENCE_LENGTH:]]).unsqueeze(0)
    return input_seq

def sample_next(predictions):
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu()
    next_token = torch.argmax(probabilities)
    return int(next_token.cpu())

def text_generator(sentence, generate_length):
    model.eval()
    sample = sentence
    for i in range(generate_length):
        int_vector = return_int_vector(sample)
        if len(int_vector) >= SEQUENCE_LENGTH:
            int_vector = int_vector[:, -SEQUENCE_LENGTH:]
        input_tensor = int_vector.to(device)
        with torch.no_grad():
            predictions = model(input_tensor)
        next_token = sample_next(predictions)
        sample += int_to_char[next_token]
    print(sample)
    print('\n')

In [40]:
# Inference for Sorting Task
def sort_numbers(input_seq):
    model.eval()
    input_seq = torch.LongTensor(input_seq).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        predictions = model(input_seq)  # Get the raw predictions
        sorted_seq = torch.argmax(predictions, dim=2).squeeze(0).long().cpu().numpy()

    return sorted(sorted_seq)

In [41]:
# Inference for sequence reversal task
def reverse_sequence(input_seq):
    model.eval()  
    # Convert input sequence to a LongTensor and add batch dimension
    input_seq = torch.LongTensor(input_seq).unsqueeze(0).to(device)  # Shape: (1, seq_length)
    
    with torch.no_grad():
        # Ensure the input shape is (batch_size, seq_len) for transformer models
        # Forward pass through the model
        predictions = model(input_seq)
        
        # Get the predicted indices
        reversed_seq = torch.argmax(predictions, dim=2).squeeze(0).long().cpu().numpy()  # Squeeze to remove the batch dimension
    return reversed_seq.tolist()

In [42]:
# Inference for copying task
def copy_sequence(input_seq):
    model.eval()  
    # Convert input sequence to a LongTensor and add batch dimension
    input_seq = torch.LongTensor(input_seq).unsqueeze(0).to(device)  # Shape: (1, seq_length)
    
    with torch.no_grad():
        # Ensure the input shape is (batch_size, seq_len) for transformer models
        # Forward pass through the model
        predictions = model(input_seq)
        
        # Get the predicted indices
        reversed_seq = torch.argmax(predictions, dim=2).squeeze(0).long().cpu().numpy()  # Squeeze to remove the batch dimension
    return reversed_seq.tolist()

In [43]:
# Inference for basic arithmetic task
def basic_arithmetic(input_seq):
    model.eval()
    input_seq = torch.LongTensor(input_seq).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        predictions = model(input_seq)  # Get the raw predictions
        result_seq = torch.argmax(predictions, dim=-1).squeeze(0).long().cpu().numpy()

        result_seq = result_seq[result_seq != 0] 
        return result_seq.tolist()  # Just return as is, without hardcoded length

In [44]:
# Example Usage:
if task_mode == 'text_gen':
    initial_text = ["Alice was"]
    generate_length = 100
    for sentence in initial_text:
        print(f"PROMPT: {sentence}")
        text_generator(sentence, generate_length)

elif task_mode == 'sort':
    input_seq = [1, 3, 2, 4, 5, 3]
    print("Original Sequence:", input_seq)
    sorted_seq = sort_numbers(input_seq)
    print("Sorted Sequence:", sorted_seq)
    
elif task_mode == 'seq_rev':
    input_seq = [1, 3, 2, 4, 5, 3]
    print("Original Sequence:", input_seq)
    reversed_seq = reverse_sequence(input_seq)
    print("Reversed Sequence:", reversed_seq)

elif task_mode == 'copy':
    input_seq = [1, 3, 2, 4, 5, 3]
    print("Input Sequence:", input_seq)
    copy_seq = copy_sequence(input_seq)
    print("Copied Sequence:", copy_seq)

elif task_mode == 'basic_arith':
    input_seq = [1, 3, 2, digit["+"], 9, 4, 2]
    print("Input Sequence:", input_seq)
    arith_seq = basic_arithmetic(input_seq)
    print("Arithmetic Sequence:", arith_seq)

Original Sequence: [1, 3, 2, 4, 5, 3]
Reversed Sequence: [3, 5, 4, 2, 3, 1]


In [45]:
from sklearn.metrics import accuracy_score

# Function to evaluate the model
def evaluate_model(model, dataloader, criterion, task_mode):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for input_seq, target_seq in dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)

            # Reshape the outputs and targets for the respective tasks
            if task_mode == 'text_gen':
                target_seq = target_seq.contiguous().view(-1)
                outputs = outputs.view(-1, vocab_size)
            
            if task_mode == 'sort' or task_mode == 'seq_rev' or task_mode == 'copy':
                target_seq = target_seq.view(-1)
                outputs = outputs.view(-1, vocab_size)

            if task_mode == 'basic_arith':
                outputs = outputs[:, :target_seq.size(1), :]
                outputs = outputs.reshape(-1, vocab_size)
                target_seq = target_seq.reshape(-1)

            loss = criterion(outputs, target_seq)
            total_loss += loss.item()

            # Get predictions
            preds = outputs.argmax(dim=1).detach().cpu().numpy()
            all_preds.extend(preds)
            all_targets.extend(target_seq.detach().cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)

    accuracy = accuracy_score(all_targets, all_preds)
    
    return avg_loss, accuracy

test_loss, test_accuracy = evaluate_model(model, test_dataloader, criterion, task_mode=task_mode)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Test Loss: 0.0070, Test Accuracy: 0.9998


In [46]:
import optuna

# Define the objective function for Optuna to optimize
def objective(trial):
    # Suggest values for the hyperparameters to tune
    num_heads = trial.suggest_int("num_heads", 2, 8)
    embed_dim = trial.suggest_int("embed_dim", num_heads * 8, num_heads * 32, step=num_heads * 4)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

    # Ensure embed_dim is divisible by num_heads
    if embed_dim % num_heads != 0:
        raise optuna.exceptions.TrialPruned()

    # Define the model with the suggested hyperparameters
    model = Transformer(
        vocab_size=vocab_size,
        embed_dim=embed_dim,
        num_layers=num_layers,
        num_heads=num_heads,
        seq_len=seq_len,
        output_size=output_size,
        dropout_rate=dropout_rate
    ).to(device)
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Training loop parameters
    epochs = 5
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0
        for input_seq, target_seq in train_dataloader:
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs = model(input_seq)
            
            # Apply the appropriate task mode setup for loss calculation
            if task_mode == 'text_gen':
                target_seq = target_seq.contiguous().view(-1)
                outputs = outputs.view(-1, vocab_size)
                loss = criterion(outputs, target_seq)
            
            if task_mode in ['sort', 'seq_rev']:
                target_seq = target_seq.view(-1)
                outputs = outputs.view(-1, vocab_size)
                loss = criterion(outputs, target_seq)

            if task_mode == 'copy':
                target_seq = target_seq.view(-1)
                outputs = outputs.view(-1, vocab_size)
                loss = criterion(outputs, target_seq)
            
            if task_mode == 'basic_arith':
                outputs = outputs[:, :target_seq.size(1), :]
                outputs = outputs.reshape(-1, vocab_size)
                target_seq = target_seq.reshape(-1)
                loss = criterion(outputs, target_seq)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Return the average loss as the metric to minimize
    epoch_loss = running_loss / len(train_dataloader)
    return epoch_loss

# Optuna study setup
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)  # Number of trials to search for optimal hyperparameters

# Print the best trial
print("Best trial:")
trial = study.best_trial
print(f"  Loss: {trial.value}")
print("  Best hyperparameters: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-11-01 00:25:46,863] A new study created in memory with name: no-name-b320dd26-e94b-4eb3-a579-be5c3a5d844a
[I 2024-11-01 00:27:21,453] Trial 0 finished with value: 1.8658884115219116 and parameters: {'num_heads': 2, 'embed_dim': 32, 'num_layers': 4, 'dropout_rate': 0.3238403198747187, 'lr': 7.428618935615208e-05}. Best is trial 0 with value: 1.8658884115219116.
[I 2024-11-01 00:27:59,076] Trial 1 finished with value: 1.6499424171447754 and parameters: {'num_heads': 7, 'embed_dim': 84, 'num_layers': 1, 'dropout_rate': 0.367626358537845, 'lr': 0.00012781657598078065}. Best is trial 1 with value: 1.6499424171447754.
[I 2024-11-01 00:28:56,970] Trial 2 finished with value: 0.7752287367582321 and parameters: {'num_heads': 2, 'embed_dim': 24, 'num_layers': 3, 'dropout_rate': 0.2130157994394271, 'lr': 0.004330901457134776}. Best is trial 2 with value: 0.7752287367582321.
[I 2024-11-01 00:32:14,939] Trial 3 finished with value: 1.2418531103134156 and parameters: {'num_heads': 7, 'embed_

Best trial:
  Loss: 0.7752287367582321
  Best hyperparameters: 
    num_heads: 2
    embed_dim: 24
    num_layers: 3
    dropout_rate: 0.2130157994394271
    lr: 0.004330901457134776
